<a href="https://colab.research.google.com/github/brunotagoncalves/publicdataml/blob/main/Estrutura%C3%A7%C3%A3o_de_Dados_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.	Estruturação de Dados
Notebook para Instalação em **Spark**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget http://www-eu.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz


--2021-02-19 12:24:13--  http://www-eu.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.26.30, 2a01:4f9:2a:1a61::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.26.30|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz [following]
--2021-02-19 12:24:13--  https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233333392 (223M) [application/x-gzip]
Saving to: ‘spark-2.4.7-bin-hadoop2.7.tgz.1’

spark-2.4.7-bin-had 100%[===================>] 222.52M  19.9MB/s    in 12s     

2021-02-19 12:24:26 (18.5 MB/s) - ‘spark-2.4.7-bin-hadoop2.7.tgz.1’ saved [2

In [3]:
!tar -xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

In [4]:
!apt-get update
!apt-get install p7zip-full
!p7zip -d -k drive/MyDrive/publicdataml/*.7z # arquivos do ftp 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
df = spark.read.format('csv').option("sep", ";").option("header", "True").load('*.txt')

In [8]:

df.count()

30189752

In [9]:
test = df.limit(10)
test.toPandas()

,competência,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,idade,horascontratuais,raçacor,sexo,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,fonte
0,202009,3,31,314930,C,2513600,-1,715615,101,5,39,44,3,1,0,1,31,0,0,0,2474.080078125,8,0,1
1,202009,3,31,316470,M,6920601,1,411005,101,7,37,40,1,3,0,1,20,0,0,0,1250,5,0,1
2,202009,5,52,521640,A,116499,-1,641015,101,5,35,44,3,1,2,3,31,0,0,0,1506,99,0,1
3,202009,2,29,290340,B,810002,1,517420,101,1,58,44,6,1,0,1,20,0,0,0,1164,1,0,1
4,202009,3,33,330285,Q,8630503,-1,411005,103,6,17,24,3,1,0,1,43,0,0,0,581.1599731445313,5,1,1
5,202009,2,27,270430,P,8520100,-1,331110,101,7,21,44,6,3,0,1,31,0,0,0,998,5,0,1
6,202009,3,35,354980,G,4711301,1,421125,101,7,20,44,3,1,0,1,20,0,0,0,1561,1,0,1
7,202009,2,29,291920,N,7820500,1,511210,111,7,25,0,3,1,0,1,20,0,1,0,6204,10,0,1
8,202009,2,29,292740,F,4221902,1,731125,101,4,32,44,3,1,0,4,20,0,0,0,2228.60009765625,9,0,1
9,202009,4,42,420910,M,7112000,-1,711320,101,4,26,44,1,1,0,1,40,0,0,0,1500,5,0,1


In [10]:
import pyspark.sql.functions as F
df = df.withColumn('idade',F.col('idade').cast('int'))
df = df.withColumn('horascontratuais',F.col('horascontratuais').cast('int'))
df = df.withColumn('salário',F.col('salário').cast('float'))
df.createOrReplaceTempView('t1')


In [11]:
df = df.filter("""`salário` BETWEEN 400 AND 100000""")

In [12]:
df = df.withColumn('faixapnad',F.expr("""Case
WHEN `salário` <= 640 THEN '1º Quintil PNAD'
WHEN `salário` <= 1045 THEN '2º Quintil PNAD'
WHEN `salário` <= 1500 THEN '3º Quintil PNAD'
WHEN `salário` <= 2700 THEN '4º Quintil PNAD'
WHEN `salário` > 2700 THEN '5º Quintil PNAD'end """))

In [13]:
df.groupby('faixapnad').count().toPandas().sort_values('faixapnad')

,faixapnad,count
1,1º Quintil PNAD,605731
0,2º Quintil PNAD,3038235
3,3º Quintil PNAD,15436386
4,4º Quintil PNAD,7842081
2,5º Quintil PNAD,2290694


In [14]:
df.groupby('competência').count().toPandas().sort_values('competência')

,competência,count
9,202001,2778415
11,202002,2863276
7,202003,3048274
6,202004,2172757
4,202005,1807117
0,202006,1851568
3,202007,2093979
10,202008,2265693
1,202009,2474192
8,202010,2717516


In [15]:
df = df.sampleBy('faixapnad',
            fractions= {
                '1º Quintil PNAD':50000/605731,
                '2º Quintil PNAD':50000/3038235,
                '3º Quintil PNAD':50000/15436386,
                '4º Quintil PNAD':50000/7842081,
                '5º Quintil PNAD':50000/2290694
            })

In [16]:

df.write.mode('overwrite').format('parquet').save('drive/MyDrive/publicdataml/sample_data_v0/')


In [17]:
spark.sql("""
SELECT `competência`
  ,AVG(`salário`)
  ,std(`salário`)
  ,min(`salário`)
  ,max(`salário`)
  ,percentile_approx(`salário`, 0.25)
  ,percentile_approx(`salário`, 0.5)
  ,percentile_approx(`salário`, 0.75)
  
FROM t1
GROUP BY `competência`
order by 1
""").toPandas()

,competência,avg(salário),stddev_samp(CAST(salário AS DOUBLE)),min(salário),max(salário),"percentile_approx(salário, CAST(0.25 AS DOUBLE), 10000)","percentile_approx(salário, CAST(0.5 AS DOUBLE), 10000)","percentile_approx(salário, CAST(0.75 AS DOUBLE), 10000)"
0,202001,2090.861297,4109.680711,400.0,99800.000000,758.780029,1163.699951,1931.369995
1,202002,1979.822024,3857.448756,400.0,99000.000000,620.000000,1108.800049,1841.670044
2,202003,1949.216910,3141.654470,400.0,99800.000000,760.000000,1197.790039,1871.510010
3,202004,2103.086958,3002.215761,400.0,99000.000000,1045.000000,1358.000000,2197.000000
4,202005,2314.977366,3993.238903,400.0,91300.000000,1045.000000,1388.000000,2484.899902
5,202006,2274.837254,3518.891451,400.0,86438.679688,1045.000000,1371.900024,2376.590088
6,202007,2182.218796,3524.773744,400.0,90000.000000,998.000000,1300.000000,2140.600098
7,202008,2203.177019,3760.510886,400.0,99800.000000,998.000000,1294.790039,2139.600098
8,202009,2183.986354,4116.083286,400.0,99800.000000,998.000000,1269.000000,2070.000000
9,202010,2168.165306,4199.440040,400.0,99800.000000,998.000000,1265.469971,2000.000000


In [18]:
import pandas as pd 
df = spark.read.format('parquet').load('drive/MyDrive/publicdataml/sample_data_v0/*')

In [19]:
dicts= {}
dicts.update({'região' :         pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=1) })
dicts.update({'uf' :             pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=2) })
dicts.update({'município' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=3) })
dicts.update({'seção' :     pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=4) })
dicts.update({'subclasse' : pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=5) })
dicts.update({'categoria' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=7) })
dicts.update({'cbo2002ocupação' :            pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=8) })
dicts.update({'graudeinstrução' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=9) })
dicts.update({'sexo' :           pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=10)})
dicts.update({'raçacor' :        pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=17)})
dicts.update({'tamestabjan' :   pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=19)})

In [20]:
for key, value in dicts.items():
  if key == 'seção': 
    value.columns = value.columns + '_' + key
    value.columns = [key,value.columns[1]]
    spd = spark.createDataFrame(value)#.createOrReplaceTempView(key.encode('ascii', 'ignore').decode('ascii'))
    df = df.withColumn(key,F.col(key)).join(spd,on=key,how='left')
  elif key in df.columns: 
    value.columns = value.columns + '_' + key
    value.columns = [key,value.columns[1]]
    spd = spark.createDataFrame(value)#.createOrReplaceTempView(key.encode('ascii', 'ignore').decode('ascii'))
    df = df.withColumn(key,F.col(key).cast('bigint')).join(spd,on=key,how='left')
  else: 
    pass
  

In [21]:
pop_brasil = pd.read_excel('drive/MyDrive/publicdataml/População Brasil.xls',sheet_name=2,header=1,dtype='object')
pop_brasil = pop_brasil.loc[:,['COD. UF','COD. MUNIC','POPULAÇÃO ESTIMADA']]

In [22]:
pop_brasil = pop_brasil.dropna()

In [23]:
pop_brasil.loc[:,'município'] = pop_brasil.loc[:,['COD. UF','COD. MUNIC']].apply(lambda x: str(x[0])+str(x[1]),axis=1).str[:6]
pop_brasil.loc[:,'pop_estimada'] = pop_brasil['POPULAÇÃO ESTIMADA']
pop_brasil.loc[pop_brasil.pop_estimada.str.contains("(|)",na=False),'pop_estimada'] = pop_brasil.loc[pop_brasil.pop_estimada.str.contains("(|)",na=False),'pop_estimada'].str.split('(',expand=True)[0]
pop_brasil.loc[:,'pop_estimada'] = pop_brasil.loc[:,'pop_estimada'].astype('int')

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [24]:
pop_brasil = pop_brasil.groupby('município')['pop_estimada'].sum().reset_index()
pop_brasil.columns= ['município','pop_estimada']

In [25]:
spd = spark.createDataFrame(pop_brasil)#.createOrReplaceTempView(key.encode('ascii', 'ignore').decode('ascii'))
df = df.join(spd,on='município',how='left')


In [26]:
df.write.mode('overwrite').format('parquet').save('drive/MyDrive/publicdataml/sample_data_v1/')

In [27]:
df = spark.read.format('parquet').load('drive/MyDrive/publicdataml/sample_data_v1/*')
df.count()

249488

In [28]:
df.limit(10).toPandas()

,município,tamestabjan,raçacor,sexo,graudeinstrução,cbo2002ocupação,categoria,subclasse,seção,uf,região,competência,saldomovimentação,idade,horascontratuais,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,indicadoraprendiz,fonte,faixapnad,Descrição_região,Descrição_uf,Descrição_município,Descrição_seção,Descrição_subclasse,Descrição_categoria,Descrição_cbo2002ocupação,Descrição_graudeinstrução,Descrição_sexo,Descrição_raçacor,Descrição_tamestabjan,pop_estimada
0,110009,6,3,1,4,784205,101,710301,B,11,1,202001,-1,27,44,0,1,32,0,0,0,1088.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias Extrativas,Extração de Minério de Ferro,"Empregado - Geral, inclusive o empregado públi...",Alimentador de Linha de Producao,6ª a 9ª Fundamental,Homem,Parda,De 50 a 99 vínculos,32695
1,110009,6,3,3,7,142105,999,5320202,H,11,1,202007,1,35,44,0,1,20,0,0,0,2000.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,"Transporte, Armazenagem e Correio",Serviços de Entrega Rápida,Não Identificado,Gerente Administrativo,Médio Completo,Mulher,Parda,De 50 a 99 vínculos,32695
2,110009,6,3,3,7,421125,999,5320202,H,11,1,202010,1,34,44,0,1,20,0,0,0,1185.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,"Transporte, Armazenagem e Correio",Serviços de Entrega Rápida,Não Identificado,Operador de Caixa,Médio Completo,Mulher,Parda,De 50 a 99 vínculos,32695
3,110009,9,1,1,2,848505,101,1012101,C,11,1,202005,-1,23,44,0,1,40,0,0,0,1164.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,Até 5ª Incompleto,Homem,Branca,De 500 a 999 vínculos,32695
4,110009,9,1,3,7,848505,101,1012101,C,11,1,202008,-1,23,44,0,1,40,0,0,0,1044.0,0,1,2º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,Médio Completo,Mulher,Branca,De 500 a 999 vínculos,32695
5,110009,9,3,1,4,848505,101,1012101,C,11,1,202011,-1,29,44,0,1,40,0,0,0,1104.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,6ª a 9ª Fundamental,Homem,Parda,De 500 a 999 vínculos,32695
6,110009,9,3,3,4,848505,101,1012101,C,11,1,202002,1,35,44,0,1,20,0,0,0,1056.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,6ª a 9ª Fundamental,Mulher,Parda,De 500 a 999 vínculos,32695
7,110009,5,3,1,3,771105,101,1610201,C,11,1,202003,1,51,44,0,1,20,0,0,0,1672.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Serrarias com Desdobramento de Madeira (Desati...,"Empregado - Geral, inclusive o empregado públi...",Marceneiro,5ª Completo Fundamental,Homem,Parda,De 20 a 49 vínculos,32695
8,110009,5,2,3,3,784205,101,1621800,C,11,1,202001,1,28,44,0,1,10,0,0,0,1045.0,0,1,2º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Fabricação de Madeira Laminada e de Chapas de ...,"Empregado - Geral, inclusive o empregado públi...",Alimentador de Linha de Producao,5ª Completo Fundamental,Mulher,Preta,De 20 a 49 vínculos,32695
9,110009,1,6,1,7,521110,101,4771704,G,11,1,202005,1,18,44,0,1,20,0,0,0,1045.0,0,1,2º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista de Medicamentos Veterinários,"Empregado - Geral, inclusive o empregado públi...",Vendedor de Comercio Varejista,Médio Completo,Homem,Não informada,Nenhum vínculo,32695


In [32]:
pddf = df.toPandas()

In [33]:
pddf.head()

,município,tamestabjan,raçacor,sexo,graudeinstrução,cbo2002ocupação,categoria,subclasse,seção,uf,região,competência,saldomovimentação,idade,horascontratuais,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,indicadoraprendiz,fonte,faixapnad,Descrição_região,Descrição_uf,Descrição_município,Descrição_seção,Descrição_subclasse,Descrição_categoria,Descrição_cbo2002ocupação,Descrição_graudeinstrução,Descrição_sexo,Descrição_raçacor,Descrição_tamestabjan,pop_estimada
0,110009,6,3,1,4,784205,101,710301,B,11,1,202001,-1,27,44,0,1,32,0,0,0,1088.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias Extrativas,Extração de Minério de Ferro,"Empregado - Geral, inclusive o empregado públi...",Alimentador de Linha de Producao,6ª a 9ª Fundamental,Homem,Parda,De 50 a 99 vínculos,32695.0
1,110009,6,3,3,7,142105,999,5320202,H,11,1,202007,1,35,44,0,1,20,0,0,0,2000.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,"Transporte, Armazenagem e Correio",Serviços de Entrega Rápida,Não Identificado,Gerente Administrativo,Médio Completo,Mulher,Parda,De 50 a 99 vínculos,32695.0
2,110009,6,3,3,7,421125,999,5320202,H,11,1,202010,1,34,44,0,1,20,0,0,0,1185.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,"Transporte, Armazenagem e Correio",Serviços de Entrega Rápida,Não Identificado,Operador de Caixa,Médio Completo,Mulher,Parda,De 50 a 99 vínculos,32695.0
3,110009,9,1,1,2,848505,101,1012101,C,11,1,202005,-1,23,44,0,1,40,0,0,0,1164.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,Até 5ª Incompleto,Homem,Branca,De 500 a 999 vínculos,32695.0
4,110009,9,1,3,7,848505,101,1012101,C,11,1,202008,-1,23,44,0,1,40,0,0,0,1044.0,0,1,2º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,Indústrias de Transformação,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,Médio Completo,Mulher,Branca,De 500 a 999 vínculos,32695.0


In [35]:
pddf.to_csv('drive/MyDrive/publicdataml/sample_data.csv')

In [42]:
!7z a drive/MyDrive/publicdataml/sample_data.7z drive/MyDrive/publicdataml/sample_data.csv


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           1 file, 105077400 bytes (101 MiB)

Creating archive: drive/MyDrive/publicdataml/sample_data.7z

Items to compress: 1

  0%      0% + drive/MyDrive/publicdataml/sample_data.csv                                                   1% + drive/MyDrive/publicdataml/sample_data.csv                                                   3% + drive/MyDrive/publicdataml/sample_data.csv                                                   4% + drive/MyDrive